In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import datetime
import numpy as np

In [2]:
raw_df = pd.read_csv('../META_1min_firstratedata.csv')

In [3]:
raw_df['timestamp'] = pd.to_datetime(raw_df['timestamp'])
raw_df['avg_price'] = (raw_df['high'] + raw_df['low']) / 2
raw_df.set_index('timestamp', inplace=True)


In [4]:
df = (raw_df.between_time('09:30', '16:00').copy())

df['cum_volume'] = df.groupby(df.index.date)['volume'].cumsum()
df['cum_price_volume'] = df.groupby(df.index.date).apply(
        lambda x: (x['avg_price'] * x['volume']).cumsum()
    ).values

df['vwap'] = df['cum_price_volume'] / df['cum_volume']

df

,open,high,low,close,volume,avg_price,cum_volume,cum_price_volume,vwap
timestamp,,,,,,,,,
2022-09-30 09:30:00,136.3500,137.06,136.0400,136.9600,349018,136.55000,349018,4.765841e+07,136.550000
2022-09-30 09:31:00,137.0374,137.73,136.9300,137.5900,186809,137.33000,535827,7.331289e+07,136.821937
2022-09-30 09:32:00,137.6299,137.98,137.0524,137.2700,108575,137.51620,644402,8.824371e+07,136.938913
2022-09-30 09:33:00,137.3300,138.39,137.3171,138.3479,148992,137.85355,793394,1.087828e+08,137.110673
2022-09-30 09:34:00,138.3200,138.67,138.1000,138.1350,143775,138.38500,937169,1.286791e+08,137.306173
...,...,...,...,...,...,...,...,...,...
2023-09-29 15:56:00,299.9400,299.94,299.3600,299.8700,143409,299.65000,16401315,4.993197e+09,304.438835
2023-09-29 15:57:00,299.8900,300.44,299.8500,300.3850,133132,300.14500,16534447,5.033156e+09,304.404262
2023-09-29 15:58:00,300.4100,300.43,300.1800,300.2400,134716,300.30500,16669163,5.073612e+09,304.371132


### Step 1: Split Data into Training and Testing Sets

In [7]:
# Get unique dates
dates = df.index.normalize().unique()

# Split dates
train_dates = dates[:int(len(dates) * 0.75)]

# Create masks
mask = df.index.normalize().isin(train_dates)
train_df = df[mask].copy()
test_df = df[~mask].copy()

test_df

,open,high,low,close,volume,avg_price,cum_volume,cum_price_volume,vwap
timestamp,,,,,,,,,
2023-07-03 09:30:00,286.70,287.00,285.52,286.410,287274,286.260,287274,8.223506e+07,286.260000
2023-07-03 09:31:00,286.52,287.09,286.00,286.860,58901,286.545,346175,9.911284e+07,286.308492
2023-07-03 09:32:00,286.85,286.88,285.80,286.585,61155,286.340,407330,1.166240e+08,286.313223
2023-07-03 09:33:00,286.71,286.71,285.80,285.850,26365,286.255,433695,1.241711e+08,286.309683
2023-07-03 09:34:00,285.82,286.50,285.68,286.175,64904,286.090,498599,1.427395e+08,286.281086
...,...,...,...,...,...,...,...,...,...
2023-09-29 15:56:00,299.94,299.94,299.36,299.870,143409,299.650,16401315,4.993197e+09,304.438835
2023-09-29 15:57:00,299.89,300.44,299.85,300.385,133132,300.145,16534447,5.033156e+09,304.404262
2023-09-29 15:58:00,300.41,300.43,300.18,300.240,134716,300.305,16669163,5.073612e+09,304.371132


## testing

In [ ]:
def prepare_features(df):
    df = df.copy()
    
    df['is_market_open'] = (df.index.hour == 9) & (df.index.minute == 30)
    # Time features
    df['time_since_open'] = (df.index.hour - 9) * 60 + (df.index.minute - 30)
    
    # Volume features
    df['volume_ratio'] = df['volume'] / df.groupby(df.index.date)['volume'].transform('sum')
    df['cum_volume_ratio'] = df.groupby(df.index.date)['volume_ratio'].cumsum()
    
    # Price-volume interaction
    df['price_volume'] = df['avg_price'] * df['volume_ratio']
    
    # Lagged features
    df['prev_price'] = df.groupby(df.index.date)['avg_price'].shift(1)
    df['price_return'] = df['avg_price'].pct_change()
    
    return df


In [ ]:
# Prepare data with volume features

train_features = prepare_features(train_df)
test_features = prepare_features(test_df)

In [ ]:
# Define features (X) and target (y)

feature_columns = ['time_since_open', 'volume_ratio', 'cum_volume_ratio', 
                  'price_volume', 'prev_price', 'price_return']
X_train = train_features[feature_columns].fillna(0)
X_test = test_features[feature_columns].fillna(0)
y_train = train_df['vwap']
y_test = test_df['vwap']

In [ ]:
# Train the model

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Make predictions

test_df['predicted_vwap'] = model.predict(X_test)

In [ ]:
# Calculate slippage in basis points

test_df['slippage_bps'] = ((test_df['predicted_vwap'] - test_df['vwap']) / test_df['vwap']) * 10000


In [ ]:
# Calculate statistics

slippage_stats = {
    'Mean Slippage (bps)': test_df['slippage_bps'].mean(),
    'Std Dev Slippage (bps)': test_df['slippage_bps'].std(),
    'Median Slippage (bps)': test_df['slippage_bps'].median(),
    'Hit Rate (within 5bps)': (abs(test_df['slippage_bps']) <= 5).mean() * 100
}

print("Slippage Statistics:")
for stat, value in slippage_stats.items():
    print(f"{stat}: {value:.2f}")

In [ ]:
# Feature importance

importances = pd.Series(model.feature_importances_, index=feature_columns)
print("\nFeature Importances:")
print(importances.sort_values(ascending=False))

In [ ]:
output_file = 'results/vwap_model_predictions_01_random_tr.csv'
test_df.to_csv(output_file, index=True)
print(f"Test results saved to {output_file}")

In [ ]:
test_df.loc['2023-09-12 09:39:00']

In [ ]:
train_df